# Basic CSV checking

Checks for executions with inconsistent results, generate a shell script to execute these

In [8]:
TIME_LIMIT = 8000
ABS_TOL = 1e-4
REL_TOL = 1e-3

# checking sanity of csv
f = open('clp-2020-orig.csv', 'r')
fn = open('clp-2020-1.csv', 'w')
nlineswe = 0
fc = open('crash.sh', 'w')
from os.path import join

ipath = '/home/haroldo/inst/miplib/2017/col-feas-stab/'

# no need to include in the string setting default parameters
def remove_default(sparam: str):
    sparam = sparam.replace('perturb-on-', '')
    return sparam
    

def str_to_param(sparam: str):
    if str(sparam) == 'nan':
        return ' -initialSolve'
    res = sparam.replace('--', 'MINUS')
    res = res.replace('-', ' ')
    res = res.replace('MINUS', ' -')
    if len(res) == 0:
        res = ' -solve'
    return res

def exec_command(inst: str, params:str):
    return './runcbcroot.sh %s.mps.gz %s' % (join(ipath, inst), str_to_param(params))

aspairs = set()

for l in f:
    cols = l.split(',')
    if len(cols) != 4 or (len(cols)==4 and len(cols[3].strip())==0):
        nlineswe += 1
        fc.write(l)
    else:
        if not l.strip():
            continue
        cols = l.split(',')
        inst = cols[0]
        cols[1] = remove_default(cols[1])
        algs = cols[1]
        if not inst.strip() or not algs.strip():
            continue
        if (inst, algs) in aspairs:
            continue
        else:
            fn.write(','.join(cols))
            aspairs.add((inst, algs))
            

f.close()
fc.close()
fn.close()
print('lines with unexpected number of columns: {}'.format(nlineswe))


lines with unexpected number of columns: 2


# Instances and features

In [9]:
import pandas as pd

dfi = pd.read_csv('features.csv')
dfi.set_index('instance', inplace=True)
dfi

,cols,rows,colsPerRow,equalities,nzEqualities,percEqualities,percNzEqualities,inequalities,nzInequalities,nz,...,percColsLeast8Nz,perccolsLeast16Nz,perccolsLeast32Nz,perccolsLeast64Nz,perccolsLeast128Nz,perccolsLeast256Nz,perccolsLeast512Nz,perccolsLeast1024Nz,perccolsLeast2048Nz,perccolsLeast4096Nz
instance,,,,,,,,,,,,,,,,,,,,,
10teams,2025.0,230.0,8.804350,135,6075.0,0,50.00000,95.0,6075.0,12150.0,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116_205_1262_66,258915.0,106427.0,2.432790,1262,171415.0,0,1.46110,105165.0,11560500.0,11731900.0,...,64.10910,62.79160,56.38450,33.340700,3.928700,0.079177,0.079177,0.000000,0.000000,0.000000
121_147_1345_33,197862.0,83077.0,2.381670,1345,66155.0,0,2.11422,81732.0,3062900.0,3129060.0,...,32.74550,31.76960,27.77490,5.632210,0.074294,0.074294,0.074294,0.000000,0.000000,0.000000
121_147_1345_33_pp,66302.0,34569.0,1.917960,1345,66155.0,0,4.93521,33224.0,1274320.0,1340470.0,...,94.36370,72.09440,6.99828,0.221713,0.221713,0.000000,0.000000,0.000000,0.000000,0.000000
22433,429.0,198.0,2.166670,198,3408.0,0,100.00000,0.0,0.0,3408.0,...,53.84620,13.51980,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yue20023.1.150,1055.0,5374.0,0.196316,190,836.0,0,6.80782,5184.0,11444.0,12280.0,...,29.95260,15.92420,11.37440,3.412320,0.189573,0.000000,0.000000,0.000000,0.000000,0.000000
z26,17937.0,850513.0,0.021090,0,0.0,0,0.00000,850513.0,1715610.0,1715610.0,...,96.25360,92.39560,85.30410,67.380300,27.574300,0.256453,0.000000,0.000000,0.000000,0.000000
zeil,70116.0,81558.0,0.859707,763,763.0,0,0.04684,80795.0,1628180.0,1628940.0,...,7.57887,7.57887,7.57887,7.577440,6.978440,4.242970,0.074163,0.012836,0.000000,0.000000


# Experiments results

In [10]:

# dataframe with execution results
dfe = pd.read_csv('clp-2020-1.csv', names=['instance', 'algsetting', 'secs', 'of'])
instances = dfe.instance.unique()
algsettings = dfe.algsetting.unique()
dfe.set_index(['instance', 'algsetting'], inplace=True)

dfe

,,secs,of
instance,algsetting,,
momentum3,scal-geo-cholesky-univ-barrier,8000.00,1.000000e+08
neos-4976951-bunnoo,dualize-1-pertv--1483-duals,50.55,3.756944e+02
gen_ip_071,dualize-1-idiot-50-primals,0.02,2.218337e+03
robot_sch_syn_plain_4_100_1605_0.2#0,pertv-208-cholesky-mumps-barrier,56.32,4.062116e+01
trdHard5600-6,idiot-7-primals,1.33,-2.485494e+04
...,...,...,...
rail03,dualp-pesteep-presolve-more-scal-equi-subs-6-psi-0.51-pertv--136-passp-66-duals,8000.00,1.000000e+08
mining,dualp-pesteep-presolve-more-scal-equi-subs-6-psi-0.51-pertv--1024-passp-66-duals,1916.42,-9.497246e+08
hgms62,dualize-0-crash-idiot7-dualp-pesteep-presolve-more-scal-equi-subs-6-psi-0.51-pertv--136-passp-66-duals,3293.79,-5.341665e+04


In [11]:
from itertools import product

# checking if all instance have their features computed
for inst in instances:
    if inst not in dfi.index:
        print('instance {} without features computed'.format(inst) )

print('algsettings: {}'.format(len(algsettings)))

# checking for missing execution
nmiss = 0
total = len(instances)*len(algsettings)
from collections import defaultdict

misi = defaultdict(list)
asi = defaultdict(list)

# generate script to run missing experiments
commands = []



for inst, algs in product(instances, algsettings):
    if (inst, algs) not in dfe.index:
        #print('{} {} missing'.format(inst, algs))
        misi[inst].append(algs)
        asi[algs].append(inst)
        nmiss += 1
        iname = join(ipath, inst)
        #print(inst + ' ' + str(algs)
        commands.append(exec_command(inst, algs))

from random import shuffle
shuffle(commands)

with open('missing.sh', 'w') as f:
    for l in commands:
        f.write(l+'\n')
    f.close()


print("from a total of: {} there are {} missing.  ({}%)".format(total, nmiss, round(100*(nmiss/total),2) ))

#print('missing exps per inst: ')
#for inst in misi.keys():
    #print('{} {}'.format(inst, len(misi[inst])))

#print('missing exps per algsetting: ')
#for algs in asi.keys():
    #print('{} {}'.format(algs, len(asi[algs])))

    
# n lines: 117629

algsettings: 150
from a total of: 149400 there are 720 missing.  (0.48%)


In [12]:
# dfr is the dataset with objective function values
dfr = pd.read_csv('relaxvalues.csv', names=['instance', 'of', 'time'])
dfr['instance'] = [inst.replace('.mps.gz', '') for inst in dfr['instance']]
dfr.drop(columns=['time'], inplace=True)
dfr.set_index('instance', inplace=True)

nfound = 0

for inst in instances:
    if inst not in dfr.index:
        print('{} not found !!!'.format(inst))
        nfound += 1

print('{} instances without relaxation values'.format(nfound))

















0 instances without relaxation values


In [13]:
# checking from wrong results
from collections import defaultdict

ff = open('clp-2020-1-final.csv', 'w')
ff.write('instance,algsetting,result\n')

ierrors = defaultdict( lambda: 0 )
for inst, algs in product(instances, algsettings):
    if (inst, algs) in dfe.index:
        of = float(dfr['of'][inst])
        res = float(dfe['of'][(inst, algs)])
        secs = float(dfe['secs'][(inst, algs)])

        res_ok = True
        absdiff = abs(res-of)
        reldiff = 0.0
        
        # checking correctness of bounds
        if secs <= TIME_LIMIT*0.999:
            if absdiff > ABS_TOL:
                minv = min(abs(res), abs(of))
                maxv = max(abs(res), abs(of))
                if minv != 0.0:
                    reldiff = (maxv-minv) / minv
                    if reldiff > REL_TOL:
                        res_ok = False
                else:
                    res_ok = False
        if res_ok:
            ff.write('{},{},{}\n'.format(inst, algs, secs))
        else:
            ff.write('{},{},{}\n'.format(inst, algs, TIME_LIMIT))
            ierrors[inst] += 1
    else:
        # result not included
        ff.write('{},{},{}\n'.format(inst, algs, TIME_LIMIT))

ff.close()

sinst = sorted(ierrors.items(), key=lambda x: x[1], reverse=True)
sinst







[('adult-max5features', 39),
 ('neos-3754224-navua', 2),
 ('comp12_3idx', 1),
 ('neos-2075418-temuka', 1)]